In [2]:
# Example for my blog post at:
# https://danijar.com/introduction-to-recurrent-networks-in-tensorflow/
import functools
import sets
import tensorflow as tf


def lazy_property(function):
    attribute = '_' + function.__name__

    @property
    @functools.wraps(function)
    def wrapper(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return wrapper


class SequenceClassification:

    def __init__(self, data, target, dropout, num_hidden=200, num_layers=3):
        self.data = data
        self.target = target
        self.dropout = dropout
        self._num_hidden = num_hidden
        self._num_layers = num_layers
        self.prediction
        self.error
        self.optimize

    @lazy_property
    def prediction(self):
        # Recurrent network.
        network = tf.contrib.rnn.GRUCell(self._num_hidden)
        network = tf.contrib.rnn.DropoutWrapper(
            network, output_keep_prob=self.dropout)
        network = tf.contrib.rnn.MultiRNNCell([network] * self._num_layers)
        #data is shaped bat
        output, _ = tf.nn.dynamic_rnn(network, self.data, dtype=tf.float32)
        
        # Select last output.
        output = tf.transpose(output, [1, 0, 2])
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        
        # Softmax layer.
        weight, bias = self._weight_and_bias(
            self._num_hidden, int(self.target.get_shape()[1]))
        prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
        return prediction

    @lazy_property
    def cost(self):
        cross_entropy = -tf.reduce_sum(self.target * tf.log(self.prediction))
        return cross_entropy

    @lazy_property
    def optimize(self):
        learning_rate = 0.003
        optimizer = tf.train.RMSPropOptimizer(learning_rate)
        return optimizer.minimize(self.cost)

    @lazy_property
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))

    @staticmethod
    def _weight_and_bias(in_size, out_size):
        weight = tf.truncated_normal([in_size, out_size], stddev=0.01)
        bias = tf.constant(0.1, shape=[out_size])
        return tf.Variable(weight), tf.Variable(bias)


def main():
    # We treat images as sequences of pixel rows.
    train, test
    _, rows, row_size = train.data.shape
    num_classes = train.target.shape[1]
    data = tf.placeholder(tf.float32, [None, rows, row_size])
    target = tf.placeholder(tf.float32, [None, num_classes])
    dropout = tf.placeholder(tf.float32)
    model = SequenceClassification(data, target, dropout)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    for epoch in range(10):
        for _ in range(100):
            batch = train.sample(10)
            sess.run(model.optimize, {
                data: batch.data, target: batch.target, dropout: 0.5})
        error = sess.run(model.error, {
            data: test.data, target: test.target, dropout: 1})
        print('Epoch {:2d} error {:3.1f}%'.format(epoch + 1, 100 * error))


if __name__ == '__main__':
#     main()
    pass

In [3]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [4]:
class myLSTM():
    def __init__(self,alpha,num_classes=8,mval=0.5):
        #         All main code here
        num_features=42
        depth = 3-1
        #         init  Weights
        self.alpha=alpha

        self.weights = tf.Variable(tf.random_uniform([hidden_units*depth, num_classes],minval=-mval,maxval=mval),name='Weights')
        #         init  X & Y
        shapeX=[None,seqLen,num_features]
        shapeY=[None,seqLen,num_classses]
        # Placeholders - Used in feed_dicts
        self.pholder_x= tf.place_holder('float',shapeX,name="InputData") 
        self.pholder_y= tf.place_holder('float',shapeY,name="LabelData")

        #Now make model here -
        with tf.name_scope('Model'):
            self.cell1 = rnn.LSTMCell(num_hidden_units)

        #The Loss function -
        with tf.name_scope('Loss'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits(logits=self.pred_y,labels=y)
        #The Grad Optimiser with minimize -
        with tf.name_scope('GradDescent'):
            self.trainer = tf.train.GradientDescentOptimizer(alpha).minimize(self.loss_fn)
        #Accuracy calcs -
        with tf.name_scope('Accuracy'):
            self.acc = tf.reduce_mean(tf.cast(
                tf.equal(
                    tf.argmax(self.y,1),
                    tf.argmax(self.pred_y,1)
                    ),
                tf.float32))
        #         So, all declared now initialize
        self.sess = tf.Session()
        self.init = tf.global_variables_initializer()
        self.sess.run(self.init) #later might move somewhere.

        # Create a summary to monitor cost tensor
        tf.summary.scalar("loss", self.cost)
        # Create a summary to monitor accuracy tensor
        tf.summary.scalar("accuracy", self.acc)
        # Merge all summaries into a single op - to pass into sess.run
        self.merged_summary_op = tf.summary.merge_all()


    def predict(self, x, y):
        result = self.sess.run(self.y_predicted, feed_dict={self.x : x, self.y : y})
        return result

    def summary_writer(logs_path):
        # op to write logs to Tensorboard
        return tf.summary_writer.FileWriter(logs_path, graph=tf.get_default_graph())
        

    def cross_validate(self, x, y):
        result = self.sess.run(self.accuracy, feed_dict={self.x : x, self.y : y})
        return result
    # first arg to sess.run is what component shoud run.
    
    def  train(self,x,y):
            
        _, c, summary = sess.run([self.trainer, self.cost, self.merged_summary_op],
                                 feed_dict={ self.pholder_x :x ,self.pholder_y:y })
        return (c,summary)


In [ ]:
training_epochs=
batch_size=
display_step=10
logs_path='/tmp/tensorflow/logs'
num_batches = y.shape[0] / batch_size
# Training cycle
summary_writer = model.summary_writer(logs_path)
for epoch in range(training_epochs):
    avg_cost = 0.

    # Loop over all batches
    for i in range(num_batches):
        batch_xs, batch_ys = #??tf.train.next_batch(batch_size)
        # Run optimization op (backprop), cost op (to get loss value)
        # and summary nodes
        c,summary = model.train(batch_xs,batch_ys)
        # Write logs at every iteration
        summary_writer.add_summary(summary, epoch * num_batches + i)
        # Compute average loss
        avg_cost += c / num_batches
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

print "Optimization Finished!"
print "Accuracy:", self.acc.eval({x: self.test_x, y: self.test_y})
